# Preparação dos dados

## Obtenção dos dados

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/TCC/house_prices/train.csv').drop("Id",axis=1)

In [ ]:
df.head()

## Limpeza de dados

### Tratamento de nulos

In [ ]:
# Verifficando as características com mais valores nulos.
top_null = df.isnull().mean().sort_values(ascending=False)*100
df_nulos = pd.DataFrame(top_null.values, index = top_null.index,
                        columns = ['percentual_nulos'])

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

# Gráfico com top 10 nulos

threshold = 10
mask = np.array(df_nulos['percentual_nulos'][0:10]) > threshold

plt.figure(figsize = (12,6))
plt.title('top 10 percentual de nulos')
ax = sns.barplot(y = df_nulos['percentual_nulos'][0:10],
            x = df_nulos.index[0:10],
            palette = np.where(mask, '#BD4B5B', '#3288BD'),
            errwidth = 0)
for i in ax.containers:
    ax.bar_label(i,)
plt.savefig('top_10_percentual_nulos.pdf')

In [ ]:
# Remoção de colunas
colunas_remocao = df_nulos[df_nulos.percentual_nulos > threshold].index
df = df.drop(colunas_remocao, axis = 1)

In [ ]:
# Remoção de registros
df = df.dropna(axis = 0)

In [ ]:
df.shape

### Tratamento de constantes

In [ ]:
threshold_constante = 90

In [ ]:
df_constantes = pd.DataFrame()
for c in df.columns:
    freq_col = df[c].value_counts(normalize=True)*100
    if ((freq_col.head(1) > threshold_constante).values[0]):
        df_constantes[c] = df[c]

In [ ]:
print("Quantidade de dados avaliados como constantes = {qte}\n"\
      .format(qte = df_constantes.shape[1]) +
      "Categóricas = {cat}\n"\
      .format(cat = df_constantes.select_dtypes(include = ['object']).shape[1])+
      "Numéricas = {num}\n"\
      .format(num = df_constantes.select_dtypes(include = ['int64']).shape[1]))


In [ ]:
# Remoção
df = df.drop(df_constantes.columns, axis = 1)
df.shape

## Verificação de multicolinearidade

In [ ]:
# Instalando biblioteca
!pip3 install phik  >/dev/null & echo 'Library Phi K Installed'

In [ ]:
# Calculo de Phi K
import phik
df_phik = df.phik_matrix()

In [ ]:
plt.figure(figsize=(34,20))
plt.title(r'Correlação Phi-K ($\varphi$ K)', fontsize=28)
# Construção de uma máscara
# Usando Booleanos pois o phi k vai de 0 a 1
# Com inteiros deu problema
mask = np.triu(np.ones_like(df_phik, dtype = bool))
sns.set(font_scale=1.5)
htm = sns.heatmap(df_phik, annot=False, mask=mask,
                  vmin=0, vmax=1, cmap='vlag',fmt='.2f',)

cbar = htm.collections[0].colorbar
cbar.ax.tick_params(labelsize=20)

plt.savefig('triu.pdf')
plt.show()

#### Aplicação do Stack para processamento do dataframe
![stack figure](https://miro.medium.com/v2/resize:fit:720/format:webp/1*DYDOif_qBEgtWfFKUDSf0Q.png)

In [ ]:
def data_clean_multicolinearity(df , phi_k_corte,target):
    # caculando o phik Para o dataFrame
    df_phik = df.phik_matrix()

    # construção de uma matriz triangular superior para servir de mascara e eliminar duplicados.
    mask = np.triu(np.ones_like(df_phik, dtype=bool))

    # Aplicação de uma mascara ao dataframe pegando apenas a triangular inferior e removendo a linha do target
    df_phi_k_mask = df_phik.mask(mask).drop(target,axis=0)

    # Aplicação do Stack
    df_phi_k_mask = df_phi_k_mask.stack()

    top_phik = df_phi_k_mask[df_phi_k_mask > phi_k_corte]

    # Transformo a lista de tuplas em uma lista
    feat_phi_k = [item for tupla in top_phik.index for item in tupla]

    # Quais são as caracteristicas que mais aparecem
    freq_feat_phik = pd.DataFrame(feat_phi_k).value_counts()

    # ########################################################################
    lst_candidatos = []

    #Percorre a lista de pares ordenados
    for t in list(top_phik.index):
        prim,seg = t[0],t[1]

        # SE a frequencia de ocorrencia do primeiro é maio que a do segundo
        # E caso o primeiro estja na lista coloca o segundo
        if (freq_feat_phik[prim] > freq_feat_phik[seg]):
            if (prim not in lst_candidatos):
                lst_candidatos.append(prim)
            else:
                lst_candidatos.append(seg)

        elif (freq_feat_phik[seg] > freq_feat_phik[prim]):
            if (seg not in lst_candidatos):
                lst_candidatos.append(seg)
            else:
                lst_candidatos.append(prim)
        else:
            if (prim not in lst_candidatos):
                lst_candidatos.append(prim)
            else:
                lst_candidatos.append(seg)

    return df.drop(lst_candidatos,axis=1)

In [ ]:
df = data_clean_multicolinearity(df , 0.9,'SalePrice')

### Categóricas Ordinais

In [ ]:
df.replace(['NA','Po','Fa','TA','Gd','Ex'], [0,1,2,3,4,5],inplace=True)

### Categóricas Não Ordinais

In [ ]:
df = pd.get_dummies(df)

In [ ]:
df.shape

# Cálculo da importância de características

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

## Holdout

In [ ]:
X, y = df.drop("SalePrice",axis=1), df["SalePrice"]

X_train, X_test, \
y_train, y_test = train_test_split(X, y,
                                test_size = 0.20)

## Random Forest Regressor

In [ ]:
# Parametros para o Túnel
param_rf= {'criterion':['squared_error',\
                        'absolute_error',\
                        'friedman_mse'],
           'max_depth':[2,3,4,5,7],
           'min_samples_split':[2,3,4,5],
           'n_estimators':[100,150,200,250,300]}

In [ ]:
# Função com Tunel de hiper parametros.
def random_forest_best_params(param,X_train,y_train):
    gs_rf = GridSearchCV(RandomForestRegressor(),
                             param, scoring='r2',
                             n_jobs=-1,cv=5)
    gs_rf.fit(X_train, y_train)
    return gs_rf.best_params_

O melhor modelo para métrica R2 Score foi :
```
{
  'criterion': 'squared_error',
  'max_depth': 7,
  'min_samples_split': 5,
  'n_estimators': 100
}
```
Para realizar o teste é só descomentar a linha a baixo:

In [ ]:
#random_forest_best_params(param_rf,X_train,y_train)

In [ ]:
randomforest_r = RandomForestRegressor(criterion = 'squared_error',
                               max_depth = 7,
                               min_samples_split = 5,
                               n_estimators = 100)

## XGBosst Regressor

In [ ]:
# Parametros para o Túnel de Params
params_xgb = {\
              'n_estimators':[100,150,200,250,300],
              'max_depth':[2,3,4,5,6,7,11],
               'eta':[0.1,0.2,0.3,0.4,0.5],
            }

In [ ]:
def xgb_best_params(params,X_train,y_train):
    gs_xgb = GridSearchCV(
        XGBRegressor(),
        params,
        scoring='r2',
        n_jobs=-1, cv=5)
    gs_xgb.fit(X_train, y_train)
    return gs_xgb.best_params_

Entre esses valores o melhor modelo foi:
```
{
    'n_estimators'= 500,
    'max_depth' = 2,
    'eta' = 0.1
}
```
Para realizar o teste é só descomentar a linha a baixo:

In [ ]:
#xgb_best_params(params_xgb,X_train,y_train)

In [ ]:
xgboost_r = XGBRegressor(n_estimators = 500,max_depth = 2, eta = 0.1)

## Validação cruzada 5-fold na métrica R2-Score

In [ ]:
rf_reg, xg_reg = randomforest_r, xgboost_r
rf_reg.fit(X_train, y_train)
xg_reg.fit(X_train, y_train)

In [ ]:
cv = 5

In [ ]:
rf_scores = cross_val_score(rf_reg, X, y, cv=cv, scoring = 'r2')

In [ ]:
xgb_scores = cross_val_score(xg_reg, X, y, cv=cv, scoring = 'r2')

In [ ]:
df_Kfold = pd.DataFrame({"Random Forest":rf_scores,"XGBoost":xgb_scores})

In [ ]:
def boxplot_sorted(df, score, title, rot=90, figsize=(10,6), fontsize=12):
    df2 = df.T
    meds = df2.median().sort_values(ascending=False)
    axes = df2[meds.index].boxplot(figsize=figsize, rot=rot, fontsize=fontsize,
                                   boxprops=dict(linewidth=4, color='cornflowerblue'),
                                   whiskerprops=dict(linewidth=4, color='cornflowerblue'),
                                   medianprops=dict(linewidth=4, color='firebrick'),
                                   capprops=dict(linewidth=4, color='cornflowerblue'),
                                   flierprops=dict(marker='o', markerfacecolor='dimgray',
                                        markersize=12, markeredgecolor='black'),
                                   return_type="axes")
    axes.set_title(title, fontsize=fontsize)
    plt.savefig(title + '.pdf')
    plt.show()

In [ ]:
boxplot_sorted(df_Kfold.T, 'r2', '5-fold in R2 Score', rot=90, figsize=(10,6), fontsize=12)

## Shap

Essa função roda o SHAP N vezes  com hold-out aleatórios e retorna uma dataframe com resultado de cada rodada.

Isto foi realizado para testar a **robustez** dos resultados do SHAP.

In [ ]:
!pip install shap >/dev/null & echo "SHAP Installed"

In [ ]:
import shap
import random

In [ ]:
def vet_rand(num):
    split = []
    count = 0
    while (count < num):
        n = float("{:.2f}".format( random.random()))
        if (n not in split) and n != 0.20 and n < 0.50 and n > 0.10:
            split.append(n)
            count = count + 1
    split.append(0.2)
    return split

In [ ]:
def evaluate_shap(model, X, y, times = 5):
    df = pd.DataFrame()
    #split = vet_rand(times)

    for t in range(times):
        # Hold-out
        X_train, X_test,\
        y_train, y_test = train_test_split(X, y, test_size = 0.2)
        #y_train, y_test = train_test_split(X, y, test_size = split[t])

        # Treinamento do modelo
        model.fit(X_train, y_train)

        # Aplicar Shap
        explainer = shap.Explainer(model, X_train)
        shap_values = explainer(X_train)

        # gerando gráficos
        if t==0:
            #analise global
            shap.plots.beeswarm(shap_values, max_display = 21,show=False)
            plt.savefig('global_shap_beeswarm.pdf', format='pdf', dpi=600, bbox_inches='tight')
            plt.show()

            shap.plots.bar(shap_values, max_display = 21, show = False)
            plt.savefig('global_shap_bar.pdf', format = 'pdf', dpi = 600, bbox_inches = 'tight')
            plt.show()

            #analise local
            shap.initjs()
            shap.plots.force(shap_values[0],matplotlib = True,show = False)
            plt.savefig('local_shap_forces.pdf', format = 'pdf', dpi = 600, bbox_inches = 'tight')
            plt.show()

            shap.plots.waterfall(shap_values[0],max_display = 15, show = False)
            plt.savefig('local_shap_waterfall.pdf', format = 'pdf', dpi = 600, bbox_inches = 'tight')
            plt.show()

            ###
            shap.initjs()
            shap.plots.force(shap_values[1],matplotlib = True,show = False)
            plt.savefig('local_shap_forces1.pdf', format = 'pdf', dpi = 600, bbox_inches = 'tight')
            plt.show()

            shap.plots.waterfall(shap_values[1],max_display = 15, show = False)
            plt.savefig('local_shap_waterfall1.pdf', format = 'pdf', dpi = 600, bbox_inches = 'tight')
            plt.show()


        # Captura das top Features com base no shap
        d = { t: [np.abs(i) for i in shap_values.values.mean(axis=0)] }
        if t == 0:
            df = pd.DataFrame(d)
        else:
            df = pd.concat([df, pd.DataFrame(d)], axis = 1)

    df = df.T
    df.columns = [i for i in shap_values.feature_names]

    return df

In [ ]:
shap_n_feat = 20
shap_times = 3

### Random Forest

In [ ]:
# redução de dimensionalidade para o random forest
shap_rf = evaluate_shap(randomforest_r, X, y, shap_times)

In [ ]:
# escolha das 15 caracteristicas globais mais importantes
candidatos_rf = list(shap_rf.mean().sort_values(ascending = False)\
 [:shap_n_feat].index)

In [ ]:
candidatos_rf

### XGBoost

In [ ]:
# redução de dimensionalidade para o XGBosst
shap_xgb = evaluate_shap(xgboost_r, X, y, shap_times)

In [ ]:
# escolha das N caracteristica mais importantes.
candidatos_xgb = list(shap_xgb.mean().sort_values(ascending = False)\
[:shap_n_feat].index)

In [ ]:
candidatos_xgb

## Redução de dimensionalidade

In [ ]:
lista = {
    'Random Forest':[randomforest_r, candidatos_rf],
    'XGBoost ':[xgboost_r,candidatos_xgb]
    }

In [ ]:
def train_and_r2(model,X_train, X_test, y_train, y_test ):
    model.fit(X_train, y_train)
    # Avaliação do modelo
    y_pred = model.predict(X_test)
    score = r2_score(y_test, y_pred)
    return score

In [ ]:
## Não mexer
def evaluate(models, X, y,label, times = 50):

    dicio={}
    lst_keys = models.keys()
    for i in lst_keys:
        dicio[i] = []
        dicio[i + ' Reduced'] = []

    for i in range(times):

        # Hold-out
        X_train, X_test, \
        y_train, y_test = train_test_split(X, y, test_size = 0.20)

        #Para cada modelo faça:
        for key in lst_keys:
            # treinar o modelo COMPLETO, avaliar e  salvar.
            model = models[key][0]
            dicio[key].append(train_and_r2(model, X_train, X_test, y_train, y_test))

            # treinar o modelo REDUZIDO, avaliar e  salvar.
            feat = models[key][1]
            dicio[key+ ' Reduced'].append(
                train_and_r2(model, X_train[feat], X_test[feat], y_train, y_test))

    return pd.DataFrame(dicio)

In [ ]:
df_r2 = evaluate(lista, X, y,'marcelo')
df_r2.describe()

In [ ]:
boxplot_sorted(df_r2.T, 'r2', 'Monte carlo CV - 50 times R2-Score', rot=90, figsize=(10,6), fontsize=12)